In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.ons.gov.uk/businessindustryandtrade/internationaltrade/datasets/internationaltradeinservicesreferencetables')
    tabs = { tab.name: tab for tab in scraper.distribution().as_databaker() }

In [2]:
tab = tabs['Table C3 2009-2012']

In [3]:
observations = tab.excel_ref('E6').expand(DOWN).expand(RIGHT).is_not_blank()

In [4]:
Year = tab.excel_ref('E4').expand(RIGHT).is_not_whitespace()

In [5]:
Flow = tab.excel_ref('E3').expand(RIGHT).is_not_blank()

In [6]:
geo1 = tab.excel_ref('A5').expand(DOWN).is_not_blank() - tab.excel_ref('A94').expand(DOWN)
geo1

{<A31 'Communications Services'>, <A59 'Merchanting and Other Trade related Services'>, <A13 'Business and Professional Services'>, <A81 'Technical services'>, <A45 'Construction Services'>, <A54 'Insurance Services'>, <A65 'Personal, Cultural and Recreational  Services'>, <A37 'Computer and Information Services'>, <A92 'Total International Trade in Services'>, <A51 'Financial Services'>, <A73 'Royalties and Licenses'>, <A88 'Other Trade  in Services'>, <A5 'Agricultural, Mining and On-site Processing Services '>}

In [7]:
geo3 = tab.excel_ref('C5').expand(DOWN).is_not_blank() - tab.excel_ref('A94').expand(DOWN)
geo3

{<C9 'Other on-site processing services'>, <C74 'Use of franchise and similar rights fees'>, <C61 'Other trade - related services'>, <C15 'Advertising'>, <C38 'Computers'>, <C21 'Market research and public opinion polling'>, <C8 'Waste treatment and depollution'>, <C17 'Public relations services'>, <C27 'Other business and professional services'>, <C19 'Other business management'>, <C7 'Mining'>, <C18 'Recruitment '>, <C40 'Publishing services'>, <C77 'Purchases and sales of other royalties and licenses'>, <C83 'Engineering'>, <C39 'News agency services'>, <C46 'Construction in the UK'>, <C32 'Postal and courier'>, <C41 'Other information provision services'>, <C66 'Audio- Visual and related services '>, <C69 'Other personal, cultural and recreational services'>, <C76 'Purchases and sales of franchises and similar rights'>, <C24 'Property management'>, <C60 'Merchanting'>, <C26 'Services between related enterprises'>, <C82 'Architectural'>, <C6 'Agricultural'>, <C20 'Legal services'>, 

In [8]:
Dimensions = [
            HDim(Year,'Year',DIRECTLY,ABOVE),
            HDim(geo1,'geo1',CLOSEST,ABOVE),
#             HDim(geo2,'geo2',DIRECTLY,LEFT),
            HDim(geo3,'geo3',DIRECTLY,LEFT),
            HDim(Flow, 'Flow',CLOSEST,LEFT),
            HDimConst('Measure Type', 'GBP Total'),
            HDimConst('Unit','gbp-million')
            ]

In [9]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [10]:
new_table = c1.topandas()

In [11]:
new_table['Year'] = pd.to_numeric(new_table['Year'], errors='coerce').fillna(0)

In [12]:
new_table['Year'] = new_table['Year'].astype(int)

In [13]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [14]:
new_table['geo3'].fillna('Total', inplace = True)

In [15]:
new_table['BOP Service'] = new_table['geo1'].fillna('') + '-' + new_table['geo3'].fillna('')

In [16]:
new_table['Flow'] = new_table['Flow'].str.lower()

In [17]:
new_table['Flow'] = new_table['Flow'].map(lambda cell:cell.replace('balances', 'balance'))

In [18]:
new_table = new_table[new_table['Value'] != '' ]

In [19]:
new_table['International Trade Basis'] = 'BOP'

In [20]:
new_table['ONS Partner Geography'] = 'Whole world'

In [21]:
new_table = new_table[['ONS Partner Geography', 'Year','Flow','BOP Service', 'International Trade Basis', 'Measure Type','Value','Unit' ]]

In [22]:
# if is_interactive():
#     SubstancetinationFolder = Path('out')
#     SubstancetinationFolder.mkdir(exist_ok=True, parents=True)
#     new_table.to_csv(SubstancetinationFolder / ('tablec31.csv'), index = False)

In [23]:
new_table.head()

,ONS Partner Geography,Year,Flow,BOP Service,International Trade Basis,Measure Type,Value,Unit
0,Whole world,2009,exports,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,4.74,gbp-million
2,Whole world,2011,exports,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,13.06,gbp-million
4,Whole world,2009,imports,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,8.612,gbp-million
6,Whole world,2011,imports,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,61.24,gbp-million
8,Whole world,2009,balance,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,-3.872,gbp-million
